In [ ]:
!pip install requests pandas

In [3]:
import pandas as pd

C:\Users\Arthur\AppData\Local\Temp\ipykernel_20896\4080736814.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Upload historical data (that we downloaded with their corresponding python script) on RDS

In [4]:
# Define the file paths
file_path_production_type = 'data/production_type.csv'
file_path_power_consumption = 'data/power_consumption.csv'
file_path_weekly_forecast = 'data/weekly_forecast.csv'


# Define the data types for each column
dtype_production_type = {'production_type': 'category'}  
dtype_power_consumption = {"status": 'category'}
# dtype_weekly_forecast = {'column3': 'category'}

# Load the dataframes with specified data types
df_production_type = pd.read_csv(file_path_production_type, parse_dates=['start_date','end_date','updated_date'])
df_power_consumption = pd.read_csv(file_path_power_consumption, dtype=dtype_power_consumption, parse_dates=['start_date','end_date','updated_date'])
df_weekly_forecast = pd.read_csv(file_path_weekly_forecast, parse_dates=['start_date','end_date','updated_date'])


In [5]:
df_production_type.keys()

Index(['start_date', 'end_date', 'updated_date', 'production_value',
       'production_type'],
      dtype='object')

In [3]:
# from sqlalchemy import create_engine

# # Create SQLAlchemy engine to connect to MySQL Database
# engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
#                        .format(user="admin",
#                                pw="admin2024",
#                                host="mysql-database.c9cwko0aaypr.eu-central-1.rds.amazonaws.com",
#                                db="rte"))

# # Convert and upload the DataFrame to MySQL
# df_production_type.to_sql('production_type_5years', con = engine, if_exists = 'append', chunksize = 1000)
# df_power_consumption.to_sql('power_consumption_5years', con = engine, if_exists = 'append', chunksize = 1000)
# df_weekly_forecast.to_sql('weekly_forecast_5years', con = engine, if_exists = 'append', chunksize = 1000)

# join theses 3 tables into one and upload onto RDS

In [4]:
print(df_production_type.columns)
print(df_power_consumption.columns)
print(df_weekly_forecast.columns)

Index(['start_date', 'end_date', 'updated_date', 'production_value',
       'production_type'],
      dtype='object')
Index(['start_date', 'end_date', 'consolidated_power_consumption_value',
       'updated_date', 'status'],
      dtype='object')
Index(['start_date', 'end_date', 'forecast_value', 'updated_date'], dtype='object')


In [3]:
df_production_type.rename(columns={'updated_date': 'production_updated_date'}, inplace=True)
df_power_consumption.rename(columns={'updated_date': 'consumption_updated_date'}, inplace=True)
df_weekly_forecast.rename(columns={'updated_date': 'forecast_updated_date'}, inplace=True)


In [4]:
# Merge the dataframes on the same 30 minutes intervall for their values
merged_df = df_power_consumption.merge(df_weekly_forecast, on=['start_date', 'end_date'], how='outer')


In [19]:
# # consumption + forecast

# from sqlalchemy import create_engine

# # Create SQLAlchemy engine to connect to MySQL Database
# engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
#                        .format(user="admin",
#                                pw="admin2024",
#                                host="mysql-database.c9cwko0aaypr.eu-central-1.rds.amazonaws.com",
#                                db="rte"))

# # Convert and upload the DataFrame to MySQL
# merged_df.to_sql('consumption_and_forecast_5years', con = engine, if_exists = 'append', chunksize = 1000)

448294

## join production but it has time deltas of 1 hour instead of 2 lines for each 30 minutes

In [5]:
df_production_type.rename(columns={'end_date': 'production_end_date'}, inplace=True)
# df_production_type = df_production_type.rename(columns={'production_start_date': 'start_date'})


In [7]:
# Merge the dataframes on start_date as it is not the same interval (30 mins vs 1 hour)
merged_df_2 = merged_df.merge(df_production_type, on='start_date', how='outer')


In [8]:
from sqlalchemy import create_engine

# Create SQLAlchemy engine to connect to MySQL Database
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}"
                       .format(user="admin",
                               pw="admin2024",
                               host="mysql-database.c9cwko0aaypr.eu-central-1.rds.amazonaws.com",
                               db="rte"))

# Convert and upload the DataFrame to MySQL
merged_df_2.to_sql('production_consumption_and_forecast_5years', con = engine, if_exists = 'append', chunksize = 1000)

2910916